# Метаклассы

Алексей Умнов https://www.youtube.com/watch?v=q08Rvcd-w9Y  
Слайды доступны по адресу: http://parallels.nsu.ru/~fat/Python/

## Классы и метаклассы

<blockquote>
"Metaclasses are deeper magic that 99% of users should never worry about.
If you wonder whether you need them, you don't (the people who actually
need them know with certainty that they need them, and don't need an
explanation about why)."
</blockquote> - Python Guru Tim Peters on metaclasses.

Краткий перевод: Метаклассы обычно использовать не нужно

Метакласс - это объект, создающий классы  
Класс     - это объект, создающий объекты

In [2]:
x = int

print(int)

<class 'int'>


Аналогично вызову: `y = int()` который возвращает объект типа `int` в переменную `y`:

In [3]:
y = x()

print(y)

0


Объект `y` принадлежит классу `int`, который его создал:

In [4]:
type(y)

int

## Метакласс type

Метакласс `type` - это самый главный метакласс

type(name, bases, attrs)
* Возвращает класс с именем `name`
* Родителями `bases`
* Атрибутами `attrs`

Можно создать класс таким способом, "на лету" (сгенерировать):

In [1]:
A = type('MyClass', (object,), {})

Тип `type` является метаклассом для класса `A`:

In [2]:
type(A)

type

Переменная `A` равна классу `MyClass`:

In [5]:
print(A)

<class '__main__.MyClass'>


Следующий код создает экземпляр класса `MyClass`. `A` это ссылка (имя переменной) на класс `MyClass`:

In [4]:
a = A()

print(a)

Создадим класс `B` с двумя атрибутами - `f1` и `f2`:

In [6]:
B = type('MyClass', (object,), {'f1': lambda: 'abc', 'f2': lambda self: 'def'})

Тип `type` является метаклассом для класса `B`:

In [7]:
type(B)

type

In [8]:
print(B)

<class '__main__.MyClass'>


Атрибуты класса:

In [9]:
B.f1

<function __main__.<lambda>>

In [10]:
B.f2

<function __main__.<lambda>>

Вызовим первый атрибут/метод класса:

In [11]:
B.f1()

'abc'

Создадим экземпляр класса `MyClass`. `B` это ссылка (имя переменной) на класс `MyClass`:

In [13]:
b = B()

print(b)

Вызовим метод экземпляра класса:

In [14]:
b.f2()

'def'

## Универсальный меткласс

In [15]:
x = 1

Вызов с одним аргументом возвращает тип объекта (класса).

`x` является объектом класса `int`:

In [16]:
type(x)

int

Класс `int` является объектом метакласса `type`:

In [17]:
type(int)

type

Класс `type` создает сам себя:

In [18]:
type(type)

type

Пример. `type` создает класс `A`, он является метаклассом для класса `A`:

In [19]:
class A(): pass

Для класса `A` является метакласс (или его тип) `type`:

In [20]:
type(A)

type

Этот анонимный объект является экземпляром класса `A`:

In [21]:
type(A())

__main__.A

## Атрибут `__class__`

Класс это ~= Тип

In [23]:
class A(): pass

a = A()

Класс `A` был создан метаклассом `type`:

In [24]:
A.__class__

type

Экземпляр класса `a` был создан классом `A`:

In [25]:
a.__class__

__main__.A

Класс `type` создал сам себя:

In [26]:
type.__class__

type

* **a**    - является экземпляром класса `A`
* **A**    - является экземпляром метакласса `type` и классом (типом) для объекта `a`
* **type** - является метаклассом

## Механизм создания классов

In [27]:
class A():

    def f(self):
        print('abc')

    x = 1

* Создаются `f`, `x`, ... (рекурсивно)
* Вызывается:
  ````A = type('A', (object,), {'f': f, 'x': x})````
* `type` делает все внутренние регистрации и т.п.
* `type` - меткласс по умолчанию

## Изменение метакласса

In [28]:
class A():
    pass

class B(type): # Собственный метакласс должен наследоваться от класса type
    pass

class C(metaclass=B):
    pass

Метаклассом для класса `A` является класс `type`:

In [29]:
type(A)

type

Метаклассом для класса `B` является класс `type`:

In [30]:
type(B)

type

Метаклассом для класса `C` является класс `B`:

In [31]:
type(C)

__main__.B

## Метаклассы в общем виде

Особые методы:
* `__new__`  - Конструирование класса/объекта
* `__init__` - Инициализация класса/объекта

Пример демонстрирует пользу изменения метакласса по умолчанию (type) на собственный, чтобы для производных классов создавалось что-то общее (атрибуты, методы). Например, когда класс `A` будет создаваться, то его метакласс `MyMeta`, который его создает, создаст в классе метод `f(self)`:

In [32]:
class MyMeta(type):

    # В этом методе класс создается и на него возвращается из этого метода ссылка
    def __new__(cls, name, bases, attrs):
        print('Called: MyMeta.__new__({}, {}, {}, {})'.format(cls.__class__.__name__, name, bases, attrs))
        return super().__new__(cls, name, bases, attrs)

    # Класс уже создан методом __new__ и сюда передается по ссылке cls вместе с другими атрибутами
    def __init__(cls, name, bases, attrs):
        print('Called: MyMeta.__init__({}, {}, {}, {})'.format(cls.__name__, name, bases, attrs))
        super().__init__(name, bases, attrs)
        # В каждом производном классе создается метод f (это атрибут класса которому
        # присваивается безымянная функция)
        cls.f = lambda self: print('qwerty')

def myfunc(name, bases, attrs):
    return MyMeta(name, bases, attrs)

class A(metaclass=MyMeta):
    pass

class B(metaclass=MyMeta):
    def f(self):
        print('Этот метод будет переопределен метаклассом MyMeta')

class C(metaclass=myfunc):
    # Можно еще указать метакласс как функцию возвращающую метакласс,
    # в которой могут быть определены какие-то особые процедуры
    pass

Called: MyMeta.__new__(type, A, (), {'__module__': '__main__', '__qualname__': 'A'})
Called: MyMeta.__init__(A, A, (), {'__module__': '__main__', '__qualname__': 'A'})
Called: MyMeta.__new__(type, B, (), {'__module__': '__main__', '__qualname__': 'B', 'f': <function B.f at 0x7fda89fc7598>})
Called: MyMeta.__init__(B, B, (), {'__module__': '__main__', '__qualname__': 'B', 'f': <function B.f at 0x7fda89fc7598>})
Called: MyMeta.__new__(type, C, (), {'__module__': '__main__', '__qualname__': 'C'})
Called: MyMeta.__init__(C, C, (), {'__module__': '__main__', '__qualname__': 'C'})


Это метакласс для класса `A`:

In [33]:
type(A)

__main__.MyMeta

Создается экземпляр класса `A`:

In [34]:
a = A()

В классе `A` появился метод, который создал метакласс `MyMeta`:

In [35]:
a.f()

qwerty


Это метакласс для класса `B`:

In [37]:
type(B)

__main__.MyMeta

Создается экземпляр класса `B`:

In [38]:
b = B()

Метакласс `MyMeta` переопределил метод `f(self)` изначально записанный в классе:

In [39]:
b.f()

qwerty


Это метакласс для класса `C`:

In [40]:
type(C)

__main__.MyMeta

Создается экземпляр класса `C`:

In [41]:
c = C()

Метакласс `MyMeta` определил метод `f(self)`:

In [42]:
c.f()

qwerty


Создаем класс `D` с помощью метакласса `MyMeta` и назначаем на него переменную `D`:

In [43]:
D = MyMeta('D', (object,), {})

Called: MyMeta.__new__(type, D, (<class 'object'>,), {})
Called: MyMeta.__init__(D, D, (<class 'object'>,), {})


In [44]:
type(D)

__main__.MyMeta

In [45]:
d = D()

d.f()

qwerty


Процесс создания класса, `type`:

Предположим, что мы пишем следующую строчку:

````python
C = type(name, bases, attrs)
````

Это будет эквивалентно следующим вызовам:
1. `type` является Callable-объектом (переопределены круглые скобки), поэтому:
  ````python
  C = type.__call__(name, bases, attrs)
  ````
2. Затем, происходит конструирование класса:
  ````python
  C = type.__new__(metacls, name, bases, attrs)
  ````
3. Инициализация класса:
  ````python
  C = type.__init__(C, name, bases, attrs)
  ````

## Метаклассы, наследование, разрешение имен

* Метаклассы наследуются от `type`
* При наследовании классов метакласс сохраняется
* Разрешение имен в классах: сам класс и его метакласс (с учетом наследования)
* Разрешение имен в экземплярах: экземпляр и его класс (с учетом наследования)

In [46]:
class Meta(type):

    def f(cls):
        print('hi, from ', __class__.__name__)


class A(metaclass=Meta):
    pass

Метод `f` класса `Meta` наследуется в классе `A`, но класс `A` на самом деле наследуется от класса `object`:

In [48]:
A.f()

hi, from  Meta


Класс `A` явно наследуется от класса `object`:

In [49]:
A.__bases__

(object,)

Метод `f` наследуется НЕявно в классе `A`. Его нет в списке:

In [50]:
'f' in dir(A)

False

In [51]:
dir(A)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__']

Также, метод `f` не доступен через экземпляр класса:

In [52]:
a = A()

a.f()

AttributeError: 'A' object has no attribute 'f'

## Пример метакласса

Задача:

Для каждого метода в классе добавлять для него короткий псевдоним (первая буква в верхнем регистре)

In [53]:
class AddShortNames(type):

    def __init__(cls, name, bases, attrs):
        super().__init__(name, bases, attrs)

        for k, v in attrs.items():
            if callable(v):
                short_name = k[0].upper()
                print('Создаем альяс {} для объекта {}'.format(short_name, v))
                setattr(cls, short_name, v)
                # Также, можно было бы
                # 1) Удалять какие-то атрибуты с помощью delattr()
                # 2) Проверять наличие каких-то атрибутов с помощью hasattr()


class A(metaclass=AddShortNames):

    def foo(self):
        print('Called foo')

    def bar(self):
        print('Called bar')

Создаем альяс F для объекта <function A.foo at 0x7fda89fabe18>
Создаем альяс B для объекта <function A.bar at 0x7fda89fabf28>


In [56]:
a = A()

a.foo()

Called foo


In [57]:
a.bar()

Called bar


In [58]:
a.F() # Метод создан динамически в метаклассе AddShortNames

Called foo


In [59]:
a.B() # Метод создан динамически в метаклассе AddShortNames

Called bar


In [60]:
print('Атрибуты callable в классе A:')

for k, v in A.__dict__.items():
    if callable(v):
        print('{: >3}: {}'.format(k, v))

Атрибуты callable в классе A:
foo: <function A.foo at 0x7fda89fabe18>
bar: <function A.bar at 0x7fda89fabf28>
  F: <function A.foo at 0x7fda89fabe18>
  B: <function A.bar at 0x7fda89fabf28>


## Абстрактные атрибуты

Запрет на создание экземпляров классов-потомков, которые не переопределяют какие-то атрибуты.

In [61]:
from abc import ABCMeta, abstractmethod # ABC = Abstract Base Classes

class Shape(metaclass=ABCMeta):

    @abstractmethod
    def area(self):
        pass